In [1]:
import pandas as pd

In [2]:
pwd = 'DoD_删后.xlsx'
df = pd.read_excel(pwd)#将CSV另存为xlsx再使用read_excel函数，read_csv会报错

In [3]:
df.head()

,Unnamed: 0,Award Number,关键词-中文,关键词-英文,Project Title,Award Abstract,Funding Agency Name（处理后）,Anticipated Award Amount,DoD Awarding Office（处理后）,Recipient Organization（处理后）,Recipient Organization（消歧后）,Last Name+First Name,PI Type,Potential Period of Performance Start,Potential Period of Performance End,时长
0,1,W9132T1820004,铅酸蓄电池；脉冲技术；铅酸蓄电池；建筑工程研究实验室；发展中心；工程研究工程师；陆军部队；研...,lead acid battery;Pulse technology;lead acid b...,Lead Acid Battery Life Extension through Viral...,The primary objective of this project is to de...,United States Navy,250000,USACE- ERDC,The Board of Trustees of The University of Ill...,The Board of Trustees of The University of Ill...,Humin Zhao,Principal Investigator,2018/8/21,2019/8/20,364
1,2,N629092112006,合作研究；Technion集团；重复使用；相对论磁控管；膨胀阴极等离子体；这种磁控管；压缩态...,collaborative study;forthe Technion group;repe...,New Ideas for Advanced Relativistic Magnetrons,The generation and transmission of pulses of m...,United States Navy,150000,"Office of Naval Research, London, England (N62...",Technion Research & Development Foundation Ltd.,Technion Research & Development Foundation Ltd.,Yakov Krasik,Principal Investigator,2021/1/1,2023/12/31,1094
2,3,N629092112018,过程中；激进模式；生物磁敏性；生理模型；磁激发序列；基本过程；前所未有的细节；丰富的复杂性；...,process whilst;radical models;biological magne...,Spin biology under optimal quantum control,"From quantum clocks, computers and sensors to ...",United States Navy,325936,"Office of Naval Research, London, England (N62...",University of Exeter,University of Exeter,Daniel Kattnig,Principal Investigator,2021/4/2,2024/12/18,1356
3,4,N629092112008,浅海地区；大陆架；这水有多深；深水能产生赤潮；不同的化合物；缺氧；m深度；厚层；近海深水区；...,shallowcoastal regions;continental shelf;howth...,SHELF SLOPE DYNAMICS IN THE SEA OF OMAN: HOW S...,"The Sultanate of Oman, as other countries in t...",United States Navy,300025,"Office of Naval Research, London, England (N62...",Sultan Qaboos University,Sultan Qaboos University,Gerd Bruss,Principal Investigator,2020/12/11,2023/12/10,1094
4,5,N629092112002,作战效能；潜在效应；改变颜色的感觉，结果；关键考虑因素；色彩视觉效果；效果相同；激光护眼；色...,operational effectiveness;potential effect;alt...,Examining colour usage in next generation disp...,There has been an increasing incidence of lase...,United States Navy,107210,"Office of Naval Research, London, England (N62...",Deakin University,Deakin University,Amanda Douglass,Principal Investigator,2020/11/5,2021/11/4,364


In [4]:
df1=df[['Award Number','关键词-英文','Project Title','Award Abstract']]

In [5]:
#df1.head()

In [6]:
len(df1)

18050

In [7]:
for i in range(len(df1)):
    df1.loc[i,'keycontent']=str(df1.loc[i,'关键词-英文']) +';'+ str(df1.loc[i,'Project Title'])+';'+str(df1.loc[i,'Award Abstract'])

C:\Users\Zhou\anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\Zhou\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [8]:
df2=df1[['Award Number','keycontent']]

In [9]:
df2=df2.dropna(axis=0, how='any')

In [10]:
len(df2)

18050

In [11]:
#导入一系列包
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
#import pyLDAvis.gensim(旧版本)
import pyLDAvis.gensim_models  # don't skip this(https://blog.csdn.net/xyy731121463/article/details/116068806)
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [12]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')#看解决办法
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [13]:
#!pip install pyLDAvis

In [14]:
file_path2='D:/files/data/stopwordlisteng.TXT'
stopworddic1=stop_words
data=[]
with open(file_path2, "r") as f:
    for line in f.readlines():
        line = line.strip('\n')  #去掉列表中每一个元素的换行符
        data.append(line)
        #print(line)
for i in range(len(data)):
    stopworddic1.append(data[i])

fh=[',','(',')','e','.','g','-']

for i in range(1000):
    fh.append(str(i))

for i in range(len(fh)):
    stopworddic1.append(fh[i])

In [15]:
data = df2.keycontent.values.tolist()

In [16]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [17]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [18]:
stop_words = stopworddic1
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(data_lemmatized[:1])

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 9), (1, 2), (2, 3), (3, 9), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 2), (10, 4), (11, 2), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 2), (18, 9), (19, 2), (20, 2), (21, 3), (22, 1), (23, 3), (24, 1), (25, 1), (26, 3), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 4), (33, 1), (34, 1)]]


In [21]:

for i in range(2,51):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=i, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.5,
                                           per_word_topics=True)
    # Compute Perplexity
    #print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)
    #b.append(coherence_lda)
    



Coherence Score:  0.3596780649279899

Coherence Score:  0.47057966495841647

Coherence Score:  0.467036714426624

Coherence Score:  0.45048787388198985

Coherence Score:  0.47442309928203014

Coherence Score:  0.49307759936834045

Coherence Score:  0.4911169513752306

Coherence Score:  0.4885718412060037

Coherence Score:  0.5013405935396337

Coherence Score:  0.49488797226305503

Coherence Score:  0.5191141150980317

Coherence Score:  0.49018831396895524

Coherence Score:  0.49653086632400584

Coherence Score:  0.5158824447513088

Coherence Score:  0.501361437999235

Coherence Score:  0.5067645684428739

Coherence Score:  0.5247978920126238

Coherence Score:  0.5080220814910723

Coherence Score:  0.5146447334857297

Coherence Score:  0.5052792861567456

Coherence Score:  0.5017700550792702

Coherence Score:  0.5089069229911596

Coherence Score:  0.5202825563101

Coherence Score:  0.49990552028120633

Coherence Score:  0.5128229902898115

Coherence Score:  0.5036722791944411

Coherenc

In [22]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=18, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.5,
                                           per_word_topics=True)

In [23]:
j=0
def topics_document_to_dataframe(topics_document, num_topics):
    global j
    j=j+1
    res = pd.DataFrame(columns=range(num_topics))
    test_2 = pd.DataFrame()  
    for i in range(18):
        test_2.loc[str(j),i]=0
    res1=test_2
    #res.index=0
    #res.ix[]
    for topic_weight in topics_document:
        #print(topic_weight)
        #print(topic_weight[1])
       # print(res)
        #print(res.ix[0, topic_weight[0]])
        res1.loc[str(j),topic_weight[0]] = topic_weight[1]
    return res1

In [24]:
from pandas.core.frame import DataFrame
texts1=DataFrame(texts)
topics = [lda_model[corpus[i]] for i in range(len(texts1))]

In [25]:
document_topic=[]
document_topic = pd.concat([topics_document_to_dataframe(topics_document[0], 18) for topics_document in topics])

In [26]:
for i in range(18):
    print(i)
    print(lda_model.print_topic(i, topn=10))

0
0.063*"model" + 0.045*"datum" + 0.027*"analysis" + 0.023*"develop" + 0.023*"approach" + 0.022*"method" + 0.015*"computational" + 0.015*"theory" + 0.013*"tool" + 0.012*"framework"
1
0.058*"chemical" + 0.040*"nuclear" + 0.026*"molecular" + 0.019*"reaction" + 0.019*"molecule" + 0.019*"gas" + 0.019*"surface" + 0.018*"chemistry" + 0.018*"product" + 0.014*"synthesis"
2
0.056*"device" + 0.031*"power" + 0.030*"energy" + 0.028*"technology" + 0.027*"low" + 0.019*"performance" + 0.018*"base" + 0.017*"circuit" + 0.016*"application" + 0.013*"efficiency"
3
0.089*"cell" + 0.038*"protein" + 0.034*"human" + 0.031*"biological" + 0.023*"brain" + 0.022*"animal" + 0.021*"disease" + 0.019*"model" + 0.019*"mechanism" + 0.015*"gene"
4
0.044*"detection" + 0.028*"sensor" + 0.026*"image" + 0.019*"technique" + 0.019*"measurement" + 0.018*"noise" + 0.017*"detect" + 0.016*"time" + 0.014*"measure" + 0.014*"resolution"
5
0.037*"simulation" + 0.030*"dynamic" + 0.029*"flow" + 0.026*"experimental" + 0.023*"condition" 

In [27]:
from pyLDAvis import gensim_models

In [28]:
data=gensim_models.prepare(lda_model,corpus,id2word,mds='pcoa',sort_topics=True)
pyLDAvis.display(data)

C:\Users\Zhou\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [29]:
document_topic=[]
document_topic = pd.concat([topics_document_to_dataframe(topics_document[0], 18) for topics_document in topics])

In [30]:
document_topic

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
18051,0.010367,0.179453,0.306507,0.024153,0.000000,0.000000,0.000000,0.000000,0.095709,0.014349,0.000000,0.127088,0.000000,0.020426,0.106365,0.011373,0.057241,0.000000
18052,0.065871,0.079921,0.194170,0.000000,0.034725,0.000000,0.000000,0.090263,0.049445,0.019451,0.047261,0.105857,0.067617,0.061214,0.034815,0.000000,0.095169,0.028160
18053,0.139761,0.051303,0.017947,0.195659,0.024688,0.013470,0.011203,0.069231,0.017308,0.019584,0.022453,0.000000,0.011220,0.358818,0.023876,0.000000,0.000000,0.000000
18054,0.000000,0.140946,0.000000,0.070166,0.000000,0.000000,0.113588,0.039800,0.035120,0.138621,0.033777,0.000000,0.031492,0.075234,0.010347,0.000000,0.000000,0.258863
18055,0.000000,0.013899,0.065022,0.000000,0.000000,0.027864,0.123742,0.053559,0.014399,0.017665,0.000000,0.029470,0.019391,0.000000,0.271083,0.109010,0.192399,0.036888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36096,0.110526,0.040984,0.151090,0.031973,0.031252,0.032644,0.031766,0.225242,0.031801,0.031252,0.032076,0.031252,0.031595,0.040580,0.031295,0.050244,0.033174,0.031252
36097,0.048788,0.015940,0.345789,0.015294,0.092317,0.017044,0.055448,0.064985,0.071061,0.015782,0.016961,0.043429,0.015793,0.065965,0.016453,0.017119,0.064438,0.017394
36098,0.226650,0.019607,0.067987,0.022535,0.020832,0.019869,0.020197,0.143832,0.102033,0.019234,0.019463,0.019645,0.022335,0.024141,0.019465,0.098745,0.114197,0.019234
36099,0.066207,0.011525,0.201469,0.011426,0.012160,0.011465,0.011819,0.039675,0.071672,0.011704,0.132773,0.011594,0.011558,0.011374,0.012084,0.117329,0.242736,0.011430


In [31]:
document_topic.to_excel('D:/files/data999/GF.xls', index=False)

C:\Users\Zhou\AppData\Local\Temp/ipykernel_14188/1727444583.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  document_topic.to_excel('D:/files/data/GF.xls', index=False)
